In [ ]:
# Program to Detect the Presence of Water

# Import PYNQ Libraries
from pynq.overlays.base import BaseOverlay
from pynq.lib import PynqMicroblaze
from pynq.lib.arduino.arduino_analog import Arduino_Analog
from pynq.lib.arduino import ARDUINO_GROVE_A1
import time

# Program the ZYNQ PL
base = BaseOverlay("base.bit")

# Instantiate the Arduino peripherals as Python objects
adc = Arduino_Analog(base.ARDUINO, ARDUINO_GROVE_A1)

# Read from the Arduino Baord ADC. Read() function return type is a list
cupVolt = adc.read()

# Print Result
if cupVolt[1] > 1:
    print("Output: There is water in the cup")
    
else:
    print("Output: There is no water in the cup")

print('Output: ', cupVolt[1])

In [ ]:
#Program to read sensor and output a message to the reader pertaining to level of liquid in the cup
# Import PYNQ Libraries
from pynq.overlays.base import BaseOverlay
from pynq.lib import PynqMicroblaze
from pynq.lib.arduino.arduino_analog import Arduino_Analog
from pynq.lib.arduino import ARDUINO_GROVE_A1
import time
import math
import matplotlib.pyplot as plt

# Program the ZYNQ PL
base = BaseOverlay("base.bit")

# Instantiate the Arduino peripherals as Python objects
adc = Arduino_Analog(base.ARDUINO, ARDUINO_GROVE_A1)
        
class techieCup:
    #Find the Minimum and Maximum values of the cup based of the voltage
    
    def liquidLevel(self):
        minv = 10
        maxv = 0

        for i in range(20):
            voltage =  adc.read()
    
            self.volt = voltage[1]
    
            time.sleep(5)
    
            self.quarter = 0
            self.half = 0
            self.threequarter = 0
        
            if self.volt > maxv:
                maxv = self.volt
            if self.volt < minv:
                minv = self.volt
                
            self.maxLev = maxv         
            self.quarter = maxv*.25
            self.half = maxv*.5
            self.threequarter = .75* maxv
            self.minLev = minv
    
            print('MOVE SENSOR AROUND TO TRAIN')
            print('0%: ', minv)
            print('25%: ', self.quarter)
            print('50%: ', self.half)
            print('75%: ', self.threequarter)
            print('100%: ', maxv)
            print('------------------')

    def cupMessage(self):
        Start = input('Start Program: ')    
        if int(Start) == 1:
            status = 0
            
            while cupVolt != self.maxLev:
                voltage =  adc.read()
    
                cupVolt = voltage[1]
    
                time.sleep(1)

                if cupVolt >= self.maxLev * 0.90 and status != 4:
                    print('Cup is overflowing')
                    status = 4
                elif cupVolt < self.maxLev * 0.90 and cupVolt >= self.threequarter and status != 3:
                    print('Time to change')
                    status = 3
                elif cupVolt < self.threequarter and cupVolt >= self.half and status != 2:
                    print('halfway there')
                    status = 2
                elif cupVolt < self.half and cupVolt >= self.quarter and status != 1:
                    print('Filling up')
                    status = 1
                elif cupVolt < self.quarter and cupVolt <= self.minLev and cupVolt >= self.minLev and status != 0:
                    print ('Cup is empty') 
                    status = 0    
        else:
            print ('Cup is trained') 
            

s = techieCup()
s.liquidLevel()
s.cupMessage()
            

           

In [ ]:
from matplotlib import image
from matplotlib import pyplot
# load image as pixel array
image = image.imread('/home/xilinx/jupyter_notebooks/Cervix_Pictures/Good/01.jpg')
# summarize shape of the pixel array
print(image.dtype)
print(image.shape)
# display the array of pixels as an image
pyplot.imshow(image)
pyplot.show()

In [ ]:
import cv2 as cv
import numpy as np
import os
from os import listdir
from os.path import isfile, join
from matplotlib import image
import matplotlib.pyplot as plt
from PIL import Image
import glob
import pathlib
from matplotlib import pyplot



Bad_dir = '/home/xilinx/jupyter_notebooks/Cervix_Pictures/Bad'
Good_dir = '/home/xilinx/jupyter_notebooks/Cervix_Pictures/Good'
    
labels = np.array([os.path.basename(Bad_dir), os.path.basename(Good_dir)])
path, dirs, files_bad = next(os.walk("/home/xilinx/jupyter_notebooks/Cervix_Pictures/Bad"))
path, dirs, files_good = next(os.walk("/home/xilinx/jupyter_notebooks/Cervix_Pictures/Good"))

input_img_path = "/home/xilinx/jupyter_notebooks/Cervix_Pictures/Bad"
output_img_path = "/home/xilinx/jupyter_notebooks/Cervix_Pictures/Bad/Resize"
trainingData = []


for input_img_path in pathlib.Path("/home/xilinx/jupyter_notebooks/Cervix_Pictures/Bad").iterdir():
    try:
        with Image.open(input_img_path) as im:
            rgb_im = im.convert('RGB')
            a = np.asarray(im)
            im = Image.fromarray(a)
            trainingData.append(im)
            pyplot.imshow(im)
            pyplot.show()
            print(f"processing file {input_img_path} done...")
    except Exception as e:
        pass
    

for input_img_path in pathlib.Path("/home/xilinx/jupyter_notebooks/Cervix_Pictures/Good").iterdir():
    try:
        with Image.open(input_img_path) as im:
            rgb_im = im.convert('RGB')
            a = np.asarray(im)
            im = Image.fromarray(a)
            pyplot.imshow(im)
            pyplot.show()
            trainingData.append(im)
            print(f"processing file {input_img_path} done...")
    except Exception as e:
        pass
    
print(len(trainingdata))

In [ ]:
trainingData = np.asmatrix(trainingData)


In [ ]:
# Train the SVM
svm = cv.ml.SVM_create()
svm.setType(cv.ml.SVM_C_SVC)
svm.setKernel(cv.ml.SVM_LINEAR)
svm.setTermCriteria((cv.TERM_CRITERIA_MAX_ITER, 100, 1e-6))
svm.train(trainingData, cv.ml.ROW_SAMPLE, labels)


In [ ]:
image = np.zeros((height, width, 3), dtype=np.uint8)

In [ ]:
# Show the training data
thickness = -1
cv.circle(image, (501,  10), 5, (  0,   0,   0), thickness)
cv.circle(image, (255,  10), 5, (255, 255, 255), thickness)
cv.circle(image, (501, 255), 5, (255, 255, 255), thickness)
cv.circle(image, ( 10, 501), 5, (255, 255, 255), thickness)

In [ ]:
thickness = 2
sv = svm.getUncompressedSupportVectors()
for i in range(sv.shape[0]):
    cv.circle(image, (int(sv[i,0]), int(sv[i,1])), 6, (128, 128, 128), thickness)